# Prepare GPS data 


In [1]:
# load GPS data
import pandas as pd
from datetime import datetime
import time
from shapely.geometry import Point
import geopandas as gpd


crs = {'init': 'epsg:4326'}
to_crs = {'init': 'epsg:3395'}


def load_gps_data_melbourne(file, crs, to_crs):
    print('loading Melbourne GPS Trajectory')
    gps_track = pd.read_csv(file, 
                            header=None, 
                            names=['timestamp', 'lat', 'lon'], 
                            skiprows=[0], 
                            delim_whitespace=True)
    gps_track['geometry'] = gps_track.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    gps_track['gps'] = gps_track['geometry']
    gps_track = gpd.GeoDataFrame(gps_track, crs=crs)
    gps_track.to_crs(to_crs, inplace=True)
    return gps_track


def load_GPS_data_Seattle(file, crs, to_crs):
    print('loading Seattle GPS Trajectory')
    gps_track = pd.read_csv(file,
                            header=None,
                            names=['Data(UTC)', 'Time(UTC)', 'lat', 'lon'],
                            skiprows=[0],
                            delim_whitespace=True)
    # string to datatime
    gps_track['datatime'] = gps_track.apply(
        lambda row: datetime.strptime(row['Data(UTC)'] + ' ' + row['Time(UTC)'], '%d-%b-%Y %H:%M:%S'), axis=1)
    gps_track.drop(['Data(UTC)', 'Time(UTC)'], axis=1, inplace=True)
    # datatime to unix time
    gps_track['timestamp'] = gps_track.apply(lambda row: time.mktime(row['datatime'].timetuple()), axis=1)
    gps_track['geometry'] = gps_track.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    gps_track = gpd.GeoDataFrame(gps_track, crs=crs)
    gps_track.to_crs(to_crs, inplace=True)
    return gps_track


gps_track = load_GPS_data_Seattle('./data/Seattle/gps_data.txt', crs, to_crs)
#gps_track = load_gps_data_melbourne('./data/Melbourne/gps_track.txt', crs, to_crs)
gps_track.head()

loading Seattle GPS Trajectory


,lat,lon,datatime,timestamp,geometry
0,47.667483,-122.107083,2009-01-17 20:27:37,1.232195e+09,POINT (-13592898.33520784 6020110.32713188)
1,47.667500,-122.107067,2009-01-17 20:27:38,1.232195e+09,POINT (-13592896.48730429 6020113.074315307)
2,47.667500,-122.107067,2009-01-17 20:27:39,1.232195e+09,POINT (-13592896.48730429 6020113.074315307)
3,47.667517,-122.107033,2009-01-17 20:27:40,1.232195e+09,POINT (-13592892.7692333 6020115.821499617)
4,47.667533,-122.106983,2009-01-17 20:27:41,1.232195e+09,POINT (-13592887.20325876 6020118.56703683)


# load road network data

In [2]:
import pandas as pd
import geopandas as gpd
import shapely.wkt
from shapely.geometry import LineString, Point
import networkx as nx


## http://wikicode.wikidot.com/get-angle-of-line-between-two-points
## angle between two points
def calculate_bearing(pt1, pt2):
    import math
    x_diff = pt2.x - pt1.x
    y_diff = pt2.y - pt1.y
    bearing = math.degrees(math.atan2(y_diff, x_diff))
    if bearing < 0:
        return bearing + 360
    else:
        return bearing

def heading_difference(A, B):
    d = max(A,B)-min(A,B)
    if d > 180:
        d=360-d
    return d


def max_speed(road_type):
    if road_type == 1:
        return 100
    elif road_type == 2:
        return 60
    elif road_type == 3:
        return 50
    elif road_type == 4:
        return 50
    elif road_type == 5:
        return 80
    elif road_type == 6:
        return 30
    else:
        return 40
    
def load_road_network(filename, crs, to_crs):
    print('loading Melbourne Road Network ...')
    road = pd.read_csv(filename, 
                        header=None, 
                        names=['Edge_ID','from', 'from lon', 'from lat', 
                               'to', 'to lon', 'to lat',
                               'length', 'road type', 'bearing'], 
                        skiprows=[0], 
                        sep=' ')
    road['geometry'] = road.apply(lambda row: 
                                  LineString([(row['from lon'], row['from lat']), (row['to lon'], row['to lat'])]), axis=1)
    road['max speed'] = road.apply(lambda row: max_speed(row['road type']), axis=1)
    road['gps'] = road['geometry']
    road = gpd.GeoDataFrame(road, crs=crs, geometry='geometry')
    # coordinate transformation
    road.to_crs(to_crs, inplace=True)
    road['bbox'] = road.apply(lambda row: row['geometry'].bounds, axis=1)
    road_graph = nx.from_pandas_edgelist(
        pd.DataFrame(road, columns=('Edge_ID', 'from', 'to', 'gps', 'geometry', 'max speed', 'length', 'bbox')), 
                                     'from', 
                                     'to', 
                                     ['Edge_ID', 'max speed', 'geometry', 'length'], 
                                     create_using=nx.MultiDiGraph())
    return road, road_graph

def update_nodes_ids(road):
    #nodes_list = []
    nodes_dict = {}
    for row_index, row in road.iterrows():        
        start_point = row['geometry'].coords[0]
        start_node_id = row['From_Node_ID']
        if nodes_dict.has_key(start_node_id):
            if nodes_dict[start_node_id][0] != start_point:
                print(start_node_id, nodes_dict[start_node_id], (start_point, row_index))
        else:
            nodes_dict[start_node_id] = (start_point, row_index)
        end_point = row['geometry'].coords[-1]
        end_node_id = row['To_Node_ID']
        if nodes_dict.has_key(end_node_id):
            if nodes_dict[end_node_id][0] != end_point:
                print(end_node_id, nodes_dict[end_node_id], (end_point, row_index))
        else:
            nodes_dict[end_node_id] = (end_point, row_index)
    same_nodes=[]
    point_id_dict = {}
    road_values = road.values
    for key, value in nodes_dict.items():
        if point_id_dict.has_key(value[0]):
            node_id = point_id_dict[value[0]]
            #if (road.iloc[value[1]]['From_Node_ID'] == key) or (road.iloc[value[1]]['To_Node_ID'] == key):
            #    same_nodes.append([value[1], key, node_id])
            from_node_id = road.iloc[value[1]]['From_Node_ID']
            to_node_id = road.iloc[value[1]]['To_Node_ID']
            if from_node_id == key:                   
                #road.iloc[same_nodes[i][0]]['From_Node_ID'] = same_nodes[i][2]
                road_values[value[1]][1] = node_id
            elif to_node_id == key:           
                #road.iloc[same_nodes[i][0]]['To_Node_ID'] = same_nodes[i][2]
                road_values[value[1]][2] = node_id
            else:
                print('Node %d is not in edge %d.'%(key, value[1]))
            #same_nodes.append([value[1], key, node_id])
        else:
            point_id_dict[value[0]] = key
    
    print len(nodes_dict), len(point_id_dict)
    column_names = ['Edge_ID', 'From_Node_ID', 'To_Node_ID', 'Two_Way', 'Speed(m/s)', 'Vertex_Count', 'geometry']
    return pd.DataFrame(road_values, columns=column_names )
    
    

def load_road_network_seattle(filename, crs, to_crs):
    """
    prepare Seattle road network data
    :param filename:
    :param crs:
    :param to_crs
    :return:
    """
    print('loading Seattle Road Network ...')
    column_names = ['Edge_ID', 'From_Node_ID', 'To_Node_ID', 'Two_Way', 'Speed(m/s)', 'Vertex_Count', 'geometry']
    road = pd.read_csv(filename, header=None, names=column_names, skiprows=[0], sep='\t')
    road['geometry'] = road.apply(lambda row: shapely.wkt.loads(row['geometry']), axis=1)    
    # update node ids
    road = update_nodes_ids(road)
    #print road.values.shape, updated_road.values.shape
    road = gpd.GeoDataFrame(road, crs=crs, geometry='geometry')
    road.to_crs(to_crs, inplace=True)
    road['length'] = road.apply(lambda row: row['geometry'].length, axis=1)
    road['bbox'] = road.apply(lambda row: row['geometry'].bounds, axis=1)
    #print len(road)
    direct_edges_list = []
    idx = 0
    for row_index, row in road.iterrows():
        direct_edges_list.append([idx, row['Edge_ID'], row['From_Node_ID'], row['To_Node_ID'],
                                  row['Speed(m/s)'], row['geometry'], row['length'], row['bbox'], 1])
        idx = idx+1
        if 1 == row['Two_Way']:
            direct_edges_list.append([idx, row['Edge_ID'], row['To_Node_ID'], row['From_Node_ID'],
                                      row['Speed(m/s)'], LineString(list(row['geometry'].coords)[::-1]),
                                     row['length'], row['bbox'], 0])
            idx = idx+1
    edges = pd.DataFrame(
        direct_edges_list,
        columns=('Edge_ID', 'osm_edge_id', 'from', 'to', 'max speed', 'geometry', 'length', 'bbox', 'from_to'))
    edges['bearing'] = \
        edges.apply(lambda edge:
                    calculate_bearing(Point(edge['geometry'].coords[0]), Point(edge['geometry'].coords[1])), axis=1)
    road_graph = nx.from_pandas_edgelist(edges,
                                         'from',
                                         'to',
                                         ['Edge_ID', 'max speed', 'geometry', 'length'],
                                         create_using=nx.MultiDiGraph())
    # print len(edges)
    # print len(direct_edges_list)
    return road_graph, gpd.GeoDataFrame(edges, crs=to_crs, geometry='geometry'), 


import time
start_time = time.time()
road_graph, edges_gpd = load_road_network_seattle('./data/Seattle/road_network.txt', crs, to_crs)
# edges_gpd, road_graph = load_road_network('./data/Melbourne/complete-osm-map/streets.txt', crs, to_crs)
print("--- %s seconds ---" % (time.time() - start_time))

loading Seattle Road Network ...
133460 130029
--- 111.860000134 seconds ---


# Trajectory segmentation

In [3]:
# OBR Calculation
from shapely.geometry import Point
def obr(track, d_error, min_num_points):
    # create initial rectangle
    clusters = []
    outliers = []
    removed = []
    anchor = [False for i in range(len(track))]
    cluster = [0,1]    
    c1 = track.iloc[cluster[0]]['geometry'].buffer(d_error).exterior
    for i in range(2, len(track)):        
        c2 = track.iloc[i]['geometry'].buffer(d_error).exterior
        mbr_new = c1.union(c2).minimum_rotated_rectangle
        possible_outliers = []
        for item in cluster[1:]:
            if not mbr_new.contains(track.iloc[item]['geometry']):
                anchor[item] = True
                possible_outliers.append(item)
        # remove outliers
        j = 0
        while j < len(possible_outliers):
            if anchor[possible_outliers[j]-1] == False and anchor[possible_outliers[j]+1] ==False:
                outliers.append(possible_outliers[j])
                cluster.remove(possible_outliers[j])
                possible_outliers.remove(possible_outliers[j])                
            else:
                j=j+1
        if len(possible_outliers) > 0:            
            # divide current cluster into two parts          
            ind = cluster.index(possible_outliers[-1])
            first_part = [cluster[k] for k in range(ind+1)]
            second_part = [cluster[k] for k in range(ind,len(cluster))]
            #print possible_outliers
            #print first_part
            #print second_part            
            #print '-----'
            if len(first_part) > min_num_points:
                clusters.append(first_part)
            else:
                removed.append(first_part)
            cluster = second_part
            cluster.append(i)
            c1 = track.iloc[cluster[0]]['geometry'].buffer(d_error).exterior 
        else:
            cluster.append(i)
    if len(cluster) > min_num_points:
        clusters.append(cluster)     
    return clusters, outliers, removed       


#d_error = 10
#clusters, outliers, removed = obr(gps_track, d_error, 2)
#len(clusters), len(outliers), len(removed)

In [4]:
def calculate_heading_difference(edges_gpd, edge_id, mbr, mbr_heading):
    import shapely
    edge_bearing = edges_gpd.iloc[edge_id]['bearing']
    a = edges_gpd.iloc[edge_id]['geometry'].difference(mbr)
    if isinstance(a, shapely.geometry.multilinestring.MultiLineString):
        p1 = shapely.geometry.Point(list(a.geoms)[0].coords[-1])
        p2 = shapely.geometry.Point(list(a.geoms)[1].coords[0])
        edge_bearing = calculate_bearing(p1, p2)
    elif isinstance(a, shapely.geometry.LineString):
        if a.coords[0] == edges_gpd.iloc[edge_id]['geometry'].coords[0] and a.coords[-1] == edges_gpd.iloc[edge_id]['geometry'].coords[-1]:
            pass
        elif a.coords[0] == edges_gpd.iloc[edge_id]['geometry'].coords[0]:
            p1 = shapely.geometry.Point(a.coords[-1])
            p2 = shapely.geometry.Point(edges_gpd.iloc[edge_id]['geometry'].coords[-1])
            edge_bearing = calculate_bearing(p1, p2)
        else:
            p1 = shapely.geometry.Point(edges_gpd.iloc[edge_id]['geometry'].coords[0])
            p2 = shapely.geometry.Point(a.coords[0])
            edge_bearing = calculate_bearing(p1, p2)
    else:
        if a.is_empty:
            pass
        else:
            print type(a)
    diff = heading_difference(mbr_heading, edge_bearing)
    return diff


def filter_edges(mbr, mbr_heading, edges_gpd, edges, a_error):    
    flag = [True for i in range(len(edges))]
    count = 0
    for i in range(len(edges)):
        diff = calculate_heading_difference(edges_gpd, edges[i], mbr, mbr_heading)
        if diff > a_error:
            flag[i] = False
            count = count+1
    new_edges = []
    for i in range(len(flag)):
        if flag[i]:
            new_edges.append(edges[i])
    return new_edges


def filter_routes(routes):
    flag = [True for i in range(len(routes))]
    for i in range(len(routes)-1):
        for j in range(i+1,len(routes)):
            if flag[i] and flag[j]:
                if len(routes[i]) < len(routes[j]):
                    if (routes[i][0] in routes[j]) and (routes[i][-1] in routes[j]):
                        ind_1 = routes[j].index(routes[i][0])
                        ind_2 = routes[j].index(routes[i][-1])
                        if ind_2-ind_1+1 == len(routes[i]):
                            flag[i] = False
                            break
                else:
                    if (routes[j][0] in routes[i]) and (routes[j][-1] in routes[i]):
                        ind_1 = routes[i].index(routes[j][0])
                        ind_2 = routes[i].index(routes[j][-1])
                        if ind_2-ind_1+1 == len(routes[j]):
                            flag[j] = False
    new_routes=[]
    for i in range(len(flag)):
        if flag[i]: 
            new_routes.append(routes[i])
    return new_routes


def filter_duplicate_routes(routes):
    flag = [True for i in range(len(routes))]
    for i in range(len(routes)-1):
        for j in range(i+1,len(routes)):
            if flag[i] or flag[j]:
                if len(routes[i]) < len(routes[j]):
                    if (routes[i][0] in routes[j]) and (routes[i][-1] in routes[j]):
                        ind_1 = routes[j].index(routes[i][0])
                        ind_2 = routes[j].index(routes[i][-1])
                        if ind_2-ind_1+1 == len(routes[i]):
                            flag[j] = False
                else:
                    if (routes[j][0] in routes[i]) and (routes[j][-1] in routes[i]):
                        ind_1 = routes[i].index(routes[j][0])
                        ind_2 = routes[i].index(routes[j][-1])
                        if ind_2-ind_1+1 == len(routes[j]):
                            flag[i] = False
    new_routes=[]
    for i in range(len(flag)):
        if flag[i]: 
            new_routes.append(routes[i])
    return new_routes


def calculate_shortest_path(road_graph, edges_gpd, start_edge_id, end_edge_id):
    route = []
    if start_edge_id == end_edge_id:
        route.append(start_edge_id)
    elif edges_gpd.iloc[start_edge_id]['to'] == edges_gpd.iloc[end_edge_id]['from']:
        route.append(start_edge_id)
        route.append(end_edge_id)
    else:
        source = edges_gpd.iloc[start_edge_id]['to']
        target = edges_gpd.iloc[end_edge_id]['from']
        try:
            #net_distance = nx.shortest_path_length(road_graph, source, target, weight='length')
            sp = nx.shortest_path(road_graph, source, target, weight='length')
        except Exception as err:
            print err
            #net_distance = 3*eu_distance
        else:    
            route.append(start_edge_id)
            for i in range(1, len(sp)):
                route.append(road_graph[sp[i-1]][sp[i]][0]['Edge_ID'])
            route.append(end_edge_id)
    return route


def save_to_file_candidate_routes(filename, candidates):
    with open(filename, 'w') as fwritter:
        for i in range(len(candidates)):
            fwritter.write('%d:\n' % i)
            candidate_routes = candidates.iloc[i]['candidate_routes']
            # print candidate_routes
            obs_prob = candidates.iloc[i]['obs_prob']
            # print obs_prob
            for j in range(len(candidate_routes)):
                for edge_id in candidate_routes[j]:
                    fwritter.write('%d, ' % edge_id)
                fwritter.write('[%f]' % obs_prob[j])
                fwritter.write('\n')
            fwritter.write('\n')

            
def save_to_file_transit_routes(filename, candidates):
    with open(filename, 'w') as fWriter:
        for idx in range(1, len(candidates)):
            fWriter.write('%d to %d:\n' % (candidates.iloc[idx]['pre_cluster'], idx))
            possible_paths = candidates.iloc[idx]['tran_routes']
            tran_prob = candidates.iloc[idx]['tran_prob']
            distances = candidates.iloc[idx]['distances']
            for i in range(len(possible_paths)):                
                for j in range(len(possible_paths[i])):                    
                    #for k in range(len(possible_paths[i][j])):                    
                    for edge_id in possible_paths[i][j]:
                        fWriter.write('%d, ' % edge_id)
                    fWriter.write('[%f, %f, %f]\n' % (distances[i][j][0], distances[i][j][1], tran_prob[i][j]))
            fWriter.write('\n') 
            
            
def map_osm_edge_id(edges_gpd, opt_route):
    opt_route_osm_edge_id = []
    for edge_id in opt_route:
        opt_route_osm_edge_id.append((edges_gpd.iloc[edge_id]['osm_edge_id'],edges_gpd.iloc[edge_id]['from_to']))
    return opt_route_osm_edge_id


def save_to_file_matching_result_seattle(filename, opt_route):
    with open(filename, 'w') as fwritter:
        fwritter.write('%d\n' % len(opt_route))
        for i in range(len(opt_route)):
            fwritter.write('%d\t%d\n' % (opt_route[i][0], opt_route[i][1]))

# Query candidate edges and routes

In [5]:
def query_edges_by_point_range(edges_gpd, point, diameter):
    from shapely.ops import nearest_points
    edge_ids = list(edges_gpd.sindex.intersection(point.buffer(diameter).exterior.bounds, objects='raw'))
    edge_id_list = []
    for edge_id in edge_ids:
        edge = edges_gpd.iloc[edge_id]
        results = nearest_points(point, edge['geometry'])
        d = point.distance(results[1])
        if d <= diameter:
            edge_id_list.append(edge['Edge_ID'])       
    return edge_id_list


def calculate_observation_probability(edges_gpd, routes, mbr, mbr_heading):
    import shapely
    obs_prob = []
    for route in routes:
        diff_sum = 0.0
        for edge_id in route:
            diff = calculate_heading_difference(edges_gpd, edge_id, mbr, mbr_heading)
            diff_sum = diff_sum + diff
        p = 1 - diff_sum/(180*len(route))
        obs_prob.append(p)
    return obs_prob


def query_candidate_routes_by_cluster(road_graph, edges_gpd, gps_track, clusters, d_error, a_error):
    results = []
    for i in range(len(clusters)):
        start_p = gps_track.iloc[clusters[i][0]]['geometry']
        end_p = gps_track.iloc[clusters[i][-1]]['geometry']              
        c1 = start_p.buffer(d_error).exterior
        c2 = end_p.buffer(d_error).exterior
        mbr = c1.union(c2).minimum_rotated_rectangle
        mbr_heading = calculate_bearing(start_p, end_p)           
        # search for candidate edges
        start_edges = query_edges_by_point_range(edges_gpd, start_p, d_error)
        end_edges = query_edges_by_point_range(edges_gpd, end_p, d_error)
        # filtering candidate edges
        start_edges = filter_edges(mbr, mbr_heading, edges_gpd, start_edges, a_error)
        end_edges = filter_edges(mbr, mbr_heading, edges_gpd, end_edges, a_error)        
        # construct candidate routes
        routes=[]
        for start in start_edges:
            for end in end_edges:
                route = calculate_shortest_path(road_graph, edges_gpd, start, end)
                if route:
                    routes.append(route)
        
        if not routes:
            if start_edges:
                for edge_id in start_edges: routes.append([edge_id])
            if end_edges:
                for edge_id in end_edges: routes.append([edge_id])
        
        if routes:
            if i < len(clusters)-1:
                routes = filter_duplicate_routes(routes)
            #else:
            #    routes = filter_routes(routes)
        else:
            #print('cluster %d does not have candidate routes!' % i)
            #print start_edges, end_edges
            pass
        obs_prob = calculate_observation_probability(edges_gpd, routes, mbr, mbr_heading)
        results.append([mbr, mbr_heading, (start_edges, end_edges), routes, obs_prob])
        column_names = ['mbr', 'mbr_heading', 'candidate_edges', 'candidate_routes', 'obs_prob']
    return pd.DataFrame(results, columns=column_names)   

In [6]:
def calculate_transit_route(road_graph, edges_gpd, gps_track, cluster, next_cluster, route, next_route):    
    new_route=[]
    net_distance = 0
    if (next_route[0] in route):
        ind1 = route.index(next_route[0])
        if len(route)-ind1 <= len(next_route):
            if route[ind1:] == next_route[0:len(route)-ind1]:
                new_route=[edge_id for edge_id in route]
                new_route.extend(next_route[len(route)-ind1:])    
                #print route, next_route, new_route    
    if not new_route:
        sp = calculate_shortest_path(road_graph, edges_gpd, route[-1], next_route[0])
        if sp:
            new_route.extend(route)
            new_route.extend(sp[1:])
            new_route.extend(next_route[1:])      
    # calculate network distance
    if new_route:
        for i in range(len(new_route)-1):
            net_distance = net_distance + edges_gpd.iloc[new_route[i]]['length']
        net_distance = net_distance + edges_gpd.iloc[new_route[-1]]['geometry'].project(gps_track.iloc[next_cluster[-1]]['geometry'])
        net_distance = net_distance - edges_gpd.iloc[new_route[0]]['geometry'].project(gps_track.iloc[cluster[0]]['geometry'])            
    return net_distance, new_route

def calculate_transit_routes(road_graph, edges_gpd, gps_track, clusters, candidates, ind_pre, ind):
    # calculate euclidean distance
    p1 = gps_track.iloc[clusters[ind_pre][0]]['geometry']
    p2 = gps_track.iloc[clusters[ind_pre][-1]]['geometry']
    eu_distance = p1.distance(p2)
    p3 = gps_track.iloc[clusters[ind][0]]['geometry']
    eu_distance = eu_distance + p2.distance(p3)
    p4 = gps_track.iloc[clusters[ind][-1]]['geometry']
    eu_distance = eu_distance + p3.distance(p4)
    routes = []
    tran_prob = []
    distances = []
    max_prob = 0
    for r1 in candidates.iloc[ind]['candidate_routes']:
        temp_routes = []
        temp_tran_prob = []
        temp_distances = []
        for r2 in candidates.iloc[ind_pre]['candidate_routes']:            
            net_distance, new_route =  calculate_transit_route(road_graph,
                                                        edges_gpd, 
                                                        gps_track, 
                                                        clusters[ind_pre], 
                                                        clusters[ind],
                                                        r2,
                                                        r1)
            if (not new_route) and (net_distance == 0):
                t_p = 0.0000001
            elif net_distance > 2*eu_distance:
                t_p = 0.0000001
            else:
                t_p = 1-abs(eu_distance-net_distance)/eu_distance
            if t_p > max_prob:
                max_prob = t_p
            temp_routes.append(new_route)
            temp_tran_prob.append(t_p)
            temp_distances.append((eu_distance, net_distance))
        routes.append(temp_routes)
        tran_prob.append(temp_tran_prob)
        distances.append(temp_distances)  
    return routes, tran_prob, distances, max_prob


def calculate_transit_routes_by_cluster(road_graph, edges_gpd, gps_track, clusters, candidates):
    pre_cluster_ids = [-1]    
    routes_list = [[]]
    tran_prob_list = [[]]  
    distances_list = [[]]
    i_pre = 0
    i = 1
    while i < len(candidates):   
        while not candidates.iloc[i]['candidate_routes']:
            pre_cluster_ids.append(-1)
            routes_list.append([])
            tran_prob_list.append([])
            distances_list.append([])
            i = i+1
            if i >= len(candidates):
                i_pre = -1
                break
            
        while i_pre >= 0:
            if not candidates.iloc[i_pre]['candidate_routes']:
                i_pre = i_pre-1
            else:
                break
        if i_pre >= 0:
            routes, tran_prob, distances, max_prob = calculate_transit_routes(road_graph, 
                                                                              edges_gpd, 
                                                                              gps_track, 
                                                                              clusters, 
                                                                              candidates, 
                                                                              i_pre, 
                                                                              i)
       
            if max_prob < 0.001: 
                #print distances
                pre_cluster_ids.append(-1)
                routes_list.append([])
                tran_prob_list.append([])
                distances_list.append([])
            else:
                pre_cluster_ids.append(i_pre)
                routes_list.append(routes)
                tran_prob_list.append(tran_prob)
                distances_list.append(distances)
            i_pre = i
            i = i+1
        else:
            if i < len(candidates):
                pre_cluster_ids.append(-1)
                routes_list.append([])
                tran_prob_list.append([])
                distances_list.append([])
                i_pre = i
                i = i+1
    print len(candidates), len(pre_cluster_ids), len(routes_list), len(tran_prob_list), len(distances_list)
    candidates['pre_cluster'] = pre_cluster_ids
    candidates['tran_routes'] = routes_list
    candidates['tran_prob'] = tran_prob_list
    candidates['distances'] = distances_list
        
    #return f, pre, possible_paths,tran_possibilities, final_cluster_id